In [47]:
import pandas as pd
data_dir = "../data/"
output_dir = "../output/"

In [58]:
thresh = 0.00001 # 1e-5
species = pd.read_csv(data_dir + "taxonomy.csv", index_col = 0).T
pathways = pd.read_csv(data_dir + "pathways.csv", index_col = 0).T
X = pd.concat([species, pathways], axis = 1) > thresh
X.head()

,s__Abiotrophia_defectiva,s__Acidaminococcus_fermentans,s__Acidaminococcus_intestini,s__Actinomyces_graevenitzii,s__Actinomyces_odontolyticus,s__Actinomyces_oris,s__Actinomyces_turicensis,s__Actinomyces_viscosus,s__Adlercreutzia_equolifaciens,s__Aggregatibacter_segnis,...,TRNA-CHARGING-PWY: tRNA charging,TRPSYN-PWY: L-tryptophan biosynthesis,TYRFUMCAT-PWY: L-tyrosine degradation I,UBISYN-PWY: superpathway of ubiquinol-8 biosynthesis (prokaryotic),UDPNACETYLGALSYN-PWY: UDP-N-acetyl-D-glucosamine biosynthesis II,UDPNAGSYN-PWY: UDP-N-acetyl-D-glucosamine biosynthesis I,URDEGR-PWY: superpathway of allantoin degradation in plants,URSIN-PWY: ureide biosynthesis,VALDEG-PWY: L-valine degradation I,VALSYN-PWY: L-valine biosynthesis
SAMD00036192,False,False,False,True,True,False,False,False,False,False,...,True,True,False,True,False,True,False,False,False,True
SAMD00036193,False,False,False,True,False,False,False,False,True,False,...,True,True,False,False,False,True,False,False,False,True
SAMD00036194,False,False,False,False,False,False,False,False,True,False,...,True,True,False,False,False,True,False,False,False,True
SAMD00036197,False,False,False,True,True,False,False,True,True,False,...,True,True,False,False,False,True,False,False,False,True
SAMD00036204,False,False,False,True,True,True,False,False,True,False,...,True,True,False,True,False,True,True,False,False,True


In [61]:
isHealthy = pd.read_csv(data_dir + "isHealthy.csv", index_col=0).T
y = np.ravel(isHealthy.values)
y

array([ True,  True, False, ...,  True,  True,  True])

In [72]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

In [63]:
classifer = LogisticRegression(n_jobs=-1)

In [68]:
import numpy as np

param_grid = {
    "penalty" : ['l1'],
    "C" : [0.001, 0.003, 0.01, 0.03, 0.1, 0.3, 1, 3, 10, 30, 100],
    "solver" : ['saga']
}

In [69]:
model = GridSearchCV(
    estimator=classifer,
    param_grid=param_grid,
    scoring="accuracy",
    verbose=10,
    n_jobs=-1, 
    cv=10 # k fold, k = 10
)

In [70]:
%%time
model.fit(X, y)

Fitting 10 folds for each of 11 candidates, totalling 110 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Batch computation too fast (0.1846s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:    3.5s
[Parallel(n_jobs=-1)]: Batch computation too slow (7.5304s.) Setting batch_size=1.
[Parallel(n_jobs=-1)]: Done  48 tasks      | elapsed:   15.5s
[Parallel(n_jobs=-1)]: Done  91 out of 110 | elapsed:   33.4s remaining:    7.0s
[Parallel(n_jobs=-1)]: Done 103 out of 110 | elapsed:   37.2s remaining:    2.5s
[Parallel(n_jobs=-1)]: Done 110 out of 110 | elapsed:   38.3s finished


CPU times: user 2.31 s, sys: 62.9 ms, total: 2.37 s
Wall time: 40.5 s


GridSearchCV(cv=10, estimator=LogisticRegression(n_jobs=-1), n_jobs=-1,
             param_grid={'C': [0.001, 0.003, 0.01, 0.03, 0.1, 0.3, 1, 3, 10, 30,
                               100],
                         'penalty': ['l1'], 'solver': ['saga']},
             scoring='accuracy', verbose=10)

In [71]:
print(f"Best score: {model.best_score_}")
print("Best parameters set:")
best_parameters = model.best_estimator_.get_params() 
for param_name in sorted(param_grid.keys()):
    print(f"\t{param_name}: {best_parameters[param_name]}")

Best score: 0.693832300439642
Best parameters set:
	C: 0.03
	penalty: l1
	solver: saga
